In [ ]:
#Setup the notebook...

# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on

# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)
    
# Scroll to bottom when there's long output
import scrolldown

# Make sure the USER variable is set correctly
import pwd
import os

In [ ]:
cd ~/

In [ ]:
# Check out the latest version of my fork of NRPy
import os
if os.path.exists("nrpytutorial"):
    !cd nrpytutorial && git pull
else:
    !git clone https://github.com/stevenrbrandt/nrpytutorial.git

In [ ]:
cd ~/nrpytutorial

In [ ]:
import os
import grid
from cactusthorn import CactusThorn
from sympy import sympify, cos, pi

# What kind of driver will this thorn use?
# Current options are Carpet and CarpetX
grid.ET_driver = "CarpetX"

In [ ]:
# Supply an arrangement name and thorn name.
thorn = CactusThorn("TestOne","WaveToyNRPy")

In [ ]:
# Declare runtime parameters and their defaults.
# For this example we will need a wave speed, as
# well as the position for the peak of our 
# initial data pulse.

wave_speed = thorn.declare_param('wave_speed',
                                 default=1,
                                 vmin=.1,
                                 vmax=100,
                                 doc="The speed of the wave")

x0 = thorn.declare_param('x0',
                         default=0,
                         vmin=-100,
                         vmax=100,
                         doc="The x pos of the wave")

y0 = thorn.declare_param('y0',
                         default=0,
                         vmin=-100,
                         vmax=100,
                         doc="The y pos of the wave")

In [ ]:
# Declare the rhs variables corresponding to the variables we are evolving
# Note that the naming convention xxx_rhs is understood by the framework.
uu_rhs, vv_rhs = thorn.register_gridfunctions("AUX", ["uu_rhs", "vv_rhs"], centering="CCC")

# Declare the grid functions we want to evolve
uu, vv = thorn.register_gridfunctions("EVOL", ["uu", "vv"], centering="CCC")

# Get the coordinates. This is done differently in Carpet and CarpetX.
# The framework will do the correct thing for both drivers.
x,y,z = thorn.get_xyz()

In [ ]:
from outputC import lhrh
import indexedexp as ixp

# _dDD describes a second derivative
# uu_dDD[0][0] is the second derivative of uu with respect to x
# The point is that we treat derivatives as symbols and don't
# worry about how they are implemented.
uu_dDD = ixp.declarerank2("uu_dDD","sym01")
uu_dD = ixp.declarerank1("uu_dD")

# The differential equations representing the wave equation
evol_eqns = [
    lhrh(lhs=uu_rhs, rhs=vv),
    lhrh(lhs=vv_rhs, rhs=wave_speed**2*(uu_dDD[0][0] + uu_dDD[1][1]))
]

In [ ]:
# Note that we can't just use constants with sympy unless we sympify them.
k = sympify(pi/20)

init_eqns = [
    lhrh(lhs=vv, rhs=sympify(0)),
    lhrh(lhs=uu, rhs=cos(k*(x-x0))**2*cos(k*(y-y0))**2),
]

In [ ]:
import NRPy_param_funcs as par

FD_order = 4
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",FD_order)

In [ ]:
# Actually create the functions

# These can be evaluated everywhere since
# they are simply functions of x and y
thorn.add_func("wave_init", 
               body=init_eqns, 
               where='everywhere',
               schedule_bin='initial',
               doc='Do the wave init',
               centering='CCC')

# These can only be evaluated in the
# interior, since the rely on finite differences.
thorn.add_func("wave_evol",
               body=evol_eqns,
               where='interior',
               schedule_bin='ODESolvers_RHS',
               doc='Do the wave evol',
               centering='CCC')

In [ ]:
import safewrite

# Files are written by cactusthorn using a utility called safewrite.
# Safewrite will only write a file (and update its modification time)
# if the contents of the file would actually change. If the verbose
# flag is set, safewrite will output a diff to the screen, showing
# what it has written.
safewrite.verbose = False

In [ ]:
home = os.environ["HOME"]

# The location of Cactus
cactus_home = os.path.join(home,"Cactus")

# Which config we are using, i.e. which directory
# under Cactus/configs where we are modifying the build
cactus_sim = "sim-gpu"

# The thornlist we are using
cactus_thornlist = os.path.join(home, "carpetx.th")

# Generate the thorn, modify the thornlist as well as the
# ThornList file under configs. Files will only be modified
# if they are updated. If the verbose flag is set in cactusthorn,
# A diff will print to the screen when the file changes.
thorn.generate(cactus_home, cactus_config=cactus_sim, cactus_thornlist=cactus_thornlist)

In [ ]:
# Kick off the rebuild of Cactus
!bash build-gpu.sh

In [ ]:
%%writefile wave2d.par
# This is the parameter file which will tell Cactus
# which thorns to execute and with what values
ActiveThorns = "WaveToyNRPy CarpetX IOUtil ODESolvers"

ODESolvers::method = "SSPRK3"
Cactus::presync_mode = "mixed-error"

Cactus::terminate = "iteration"
$blocksize=10
$nblocks=20
Cactus::cctk_itlast = $nblocks*$blocksize

CarpetX::periodic_x = false
CarpetX::periodic_y = false

CarpetX::max_tile_size_x = 200
CarpetX::max_tile_size_y = 200
CarpetX::max_tile_size_z = 200
CarpetX::max_grid_size_x = 500
CarpetX::max_grid_size_y = 500
CarpetX::max_grid_size_z = 500

CarpetX::verbose = no
CarpetX::poison_undefined_values = no

CarpetX::xmin = -10.0
CarpetX::ymin = -10.0
CarpetX::zmin = -1.0

CarpetX::xmax = 10.0
CarpetX::ymax = 10.0
CarpetX::zmax = 1.0

CarpetX::ncells_x = 120
CarpetX::ncells_y = 120
CarpetX::ncells_z = 1

CarpetX::blocking_factor_x = 8
CarpetX::blocking_factor_y = 8
CarpetX::blocking_factor_z = 1

CarpetX::ghost_size_x = 2
CarpetX::ghost_size_y = 2
CarpetX::ghost_size_z = 0

IO::out_dir = $parfile
IO::out_every = $blocksize
IO::out_mode = "np"
IO::out_proc_every = 1
CarpetX::out_openpmd_vars = "all"

In [ ]:
# Run the code! Note that {cactus_home}, etc. get substituted.
!(cd {cactus_home} && rm -fr wave2d && ./exe/cactus_{cactus_sim} {os.getcwd()}/wave2d.par)

In [ ]:
# These are the output files generated by the run
!ls ~/Cactus/wave2d

In [ ]:
%%writefile plot-data.py
# A custom plotting function that uses openpmd to extract data
# and generate 2d color plots
import numpy as np
import matplotlib
import os
matplotlib.use("Agg")
import matplotlib.pyplot as plt
home = os.environ["HOME"]
out_dir = os.path.join(home,"Cactus","wave2d")
os.chdir(out_dir)
import openpmd_api as io
series = io.Series("wave2d.it%08T.bp", io.Access.read_only)
print("openPMD version: ", series.openPMD)
if series.contains_attribute("author"):
    print("Author: ",series.author)
for gf in ["wavetoynrpy_uugf_rl00"]:
    print(gf)
    frame = 0
    for index in series.iterations:
        i = series.iterations[index]
        #for k in i.meshes:
        #    print(k)
        uu = i.meshes[gf]
        data = None
        data_index = 0
        for k in uu:
            data = uu[k].load_chunk()
        series.flush()
        print(index,data.shape,np.max(data[data_index]),np.min(data[data_index]))
        xv = np.linspace(0,1,data[data_index].shape[0])
        yv = np.linspace(0,1,data[data_index].shape[1])
        x = np.zeros(data[data_index].shape)
        y = np.zeros(data[data_index].shape)
        for i in range(xv.shape[0]):
            x[i,:] = xv[i]
        for j in range(yv.shape[0]):
            y[:,j] = yv[j]
        plt.pcolor(x,y,data[data_index],vmin=-1,vmax=1)
        plt.savefig(f"{home}/wave%05d.png" % frame)
        frame += 1

In [ ]:
# Clear out old plot data, if any
!rm -f ../wave*.png

In [ ]:
# Generate the image files
!python3 plot-data.py

In [ ]:
!(cd ~/ && ls -F wave*)

In [ ]:
# Show the first image
# Note that there's some noise on the side. This is an output artifiact (and a bug).

from IPython.display import Image

Image("../wave00000.png")

In [ ]:
# Generate a movie
!ffmpeg -y -i ~/wave%05d.png output.gif

In [ ]:
# Show the movie

from IPython.display import Image

Image("output.gif")

In [ ]:
%%writefile prnorms.py
# This short script will grab the min/max files from the norms
# that carpetx generates during the run
import os
for i in range(1000):
    fname = "../Cactus/wave2d/norms.it%08d.tsv" % i
    if not os.path.exists(fname):
        continue
    with open(fname, "r") as fd:
        for line in fd.readlines():
            cols = line.split('\t')
            it = cols[0]
            tm = cols[1]
            vn = cols[2]
            mn = cols[3]
            mx = cols[4]
            if 'uuGF' in vn:
                print(it,tm,vn,mn,mx)

In [ ]:
# It's clear that the large values at the boundary in the openPMD output are not real.
!python3 prnorms.py

Exercises:

(1) Change the FD order from 4 to 6 and regenerate the plots

(2) The equation we evolved was

$\frac{\partial}{\partial_t} u = v$

$\frac{\partial}{\partial_t} v = c^2 \left(\frac{\partial^2}{\partial^2_x} u + \frac{\partial^2}{\partial^2_y} u \right)$

Modify the second equation to look like this:

$\frac{\partial}{\partial_t} v = c^2 \left(\frac{\partial^2}{\partial^2_x} u + \frac{\partial^2}{\partial^2_y} u \right) + \frac{\partial}{\partial_x} u$

(3) Try changing the initial data from

$\cos\left(k (x-x_0)\right)^2 \cos\left(k (y-y_0)\right)^2$

to


$\cos\left(k (x-x_0)\right)^2 \cos\left(2 k (y-y_0)\right)^2$